<a href="https://colab.research.google.com/github/aimenSaf/TimeTable-Generator-using-Genetic-AI/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

number_of_courses = 5

class Gene:
    def __init__(self, course_id, is_lab, section_id, section_strength, professor_id,
                 first_lecture_day, first_lecture_timeSlot, first_lecture_room,
                 first_lecture_room_size, second_lecture_day, second_lecture_time_slot,
                 second_lecture_room, second_lecture_room_size):
        self.course_id = course_id
        self.is_lab = is_lab
        self.section_id = section_id
        self.section_strength = section_strength
        self.professor_id = professor_id
        self.first_lecture_day = first_lecture_day
        self.first_lecture_timeSlot = first_lecture_timeSlot
        self.first_lecture_room = first_lecture_room
        self.first_lecture_room_size = first_lecture_room_size
        self.second_lecture_day = second_lecture_day
        self.second_lecture_time_slot = second_lecture_time_slot
        self.second_lecture_room = second_lecture_room
        self.second_lecture_room_size = second_lecture_room_size

    def to_binary(self):
        course_id_binary = f'{self.course_id:03b}'
        is_lab_binary = f'{int(self.is_lab)}'
        section_id_binary = f'{self.section_id:03b}'
        section_strength_binary = f'{self.section_strength:06b}'
        professor_id_binary = f'{self.professor_id:04b}'
        first_lecture_day_binary = f'{["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"].index(self.first_lecture_day):03b}'
        first_lecture_timeSlot_binary = f'{["8:30", "10:00", "11:30", "1:00", "2:30", "3:55"].index(self.first_lecture_timeSlot):03b}' if self.is_lab == 0 else f'{["8:30", "10:00", "11:30", "1:00", "2:30"].index(self.first_lecture_timeSlot):03b}'
        first_lecture_room_binary = f'{self.first_lecture_room:05b}'
        first_lecture_room_size_binary = f'{self.first_lecture_room_size:03b}'
        second_lecture_day_binary = f'{["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"].index(self.second_lecture_day):03b}'
        second_lecture_time_slot_binary = f'{["8:30", "10:00", "11:30", "1:00", "2:30", "3:55"].index(self.second_lecture_time_slot):03b}' if self.is_lab == 0 else f'{["8:30", "10:00", "11:30", "1:00", "2:30"].index(self.second_lecture_time_slot):03b}'
        second_lecture_room_binary = f'{self.second_lecture_room:07b}'
        second_lecture_room_size_binary = f'{self.second_lecture_room_size:03b}'
        binary_representation = (course_id_binary + is_lab_binary + section_id_binary +
                                 section_strength_binary + professor_id_binary +
                                 first_lecture_day_binary + first_lecture_timeSlot_binary +
                                 first_lecture_room_binary + first_lecture_room_size_binary +
                                 second_lecture_day_binary + second_lecture_time_slot_binary +
                                 second_lecture_room_binary + second_lecture_room_size_binary)
        return binary_representation

    def __str__(self):
        return (f"Course ID: {self.course_id}, Lab: {'Yes' if self.is_lab else 'No'}, Section: {self.section_id}, "
                f"Strength: {self.section_strength}, Professor ID: {self.professor_id}, "
                f"1st Lecture: {self.first_lecture_day} at {self.first_lecture_timeSlot}, Room: {self.first_lecture_room} (Size: {self.first_lecture_room_size}), "
                f"2nd Lecture: {self.second_lecture_day} at {self.second_lecture_time_slot}, Room: {self.second_lecture_room} (Size: {self.second_lecture_room_size})")

class Timetable:
    def __init__(self):
        self.genes = []
        self.binary_genes = []

    def calculate_fitness(self):
        fitness = 0  # start with zero
        room_usage = {}
        professor_courses = {}
        section_courses = {}
        professor_timeslots = {}
        room_assignments = {}

        for gene in self.genes:
            # check room capacity and room assignment
            if gene.first_lecture_room_size < gene.section_strength or gene.second_lecture_room_size < gene.section_strength:
                fitness -= 1

            # time slots and room usage checks
            timeslots = [(gene.first_lecture_day, gene.first_lecture_timeSlot, gene.first_lecture_room),
                         (gene.second_lecture_day, gene.second_lecture_time_slot, gene.second_lecture_room)]

            for day, timeslot, room in timeslots:
                if (room, day, timeslot) in room_usage:
                    fitness -= 1
                else:
                    room_usage[(room, day, timeslot)] = True

            # professor scheduling checks
            if gene.professor_id not in professor_courses:
                professor_courses[gene.professor_id] = []
                professor_timeslots[gene.professor_id] = []
            professor_courses[gene.professor_id].append(gene.course_id)
            professor_timeslots[gene.professor_id].extend(timeslots)

            # section assignment checks
            if gene.section_id not in section_courses:
                section_courses[gene.section_id] = []
            section_courses[gene.section_id].append(gene.course_id)

            # sheck for lab and theory timings
            if gene.is_lab and gene.first_lecture_timeSlot in ["8:30", "10:00", "11:30"]:
                fitness -= 1
            if not gene.is_lab and gene.first_lecture_timeSlot in ["1:00", "2:30", "3:55"]:
                fitness -= 1

            # Same room for a particular section and floor change minimization
            floor_number = (gene.first_lecture_room - 1) // 10 + 1
            if gene.section_id not in room_assignments:
                room_assignments[gene.section_id] = {}
            if gene.course_id not in room_assignments[gene.section_id]:
                room_assignments[gene.section_id][gene.course_id] = (floor_number, gene.first_lecture_room)
            else:
                assigned_floor, assigned_room = room_assignments[gene.section_id][gene.course_id]
                if assigned_room != gene.first_lecture_room:
                    fitness -= 1
                if assigned_floor != floor_number:
                    fitness -= 1

        # Professor and section course limit checks
        for courses in professor_courses.values():
            if len(set(courses)) > 3:
                fitness -= 1
        for courses in section_courses.values():
            if len(set(courses)) > 5:
                fitness -= 1

        # Adjacent day constraint
        for gene in self.genes:
            day1 = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"].index(gene.first_lecture_day)
            day2 = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"].index(gene.second_lecture_day)
            if abs(day1 - day2) <= 1:
                fitness -= 1

        return fitness

    def add_gene(self, gene):
        self.genes.append(gene)
        binary_gene = gene.to_binary()
        self.binary_genes.append(binary_gene)

def initialize_population(size):
    population = []
    for _ in range(size):
        timetable = Timetable()
        for course_id in range(number_of_courses):
            is_lab = random.choice([True, False])
            section_id = random.randint(1,5)
            section_strength = random.randint(20, 60)
            professor_id = random.randint(1, 10)
            first_lecture_day = random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
            if is_lab == False:
                first_lecture_timeSlot = random.choice(["8:30", "10:00", "11:30", "1:00", "2:30", "3:55"])
            else:
                first_lecture_timeSlot = random.choice(["8:30", "10:00", "11:30", "1:00", "2:30"])
            first_lecture_room = random.randint(1, 100)
            first_lecture_room_size = 120 if is_lab else 60
            second_lecture_day = random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
            if is_lab == False:
                second_lecture_time_slot = random.choice(["8:30", "10:00", "11:30", "1:00", "2:30", "3:55"])
            else:
                second_lecture_time_slot = random.choice(["8:30", "10:00", "11:30", "1:00", "2:30"])
            second_lecture_room = random.randint(1, 100)
            second_lecture_room_size = 120 if is_lab else 60
            gene = Gene(course_id, is_lab, section_id, section_strength, professor_id,
                        first_lecture_day, first_lecture_timeSlot, first_lecture_room, first_lecture_room_size,
                        second_lecture_day, second_lecture_time_slot, second_lecture_room, second_lecture_room_size)
            timetable.add_gene(gene)
        population.append(timetable)
        print("Generated Timetable:")
        for gene in timetable.binary_genes:
            print(f"Binary Gene: {gene}")
    return population

def select_parents(population):
    selected = random.sample(population, 2)
    return selected

def crossover(parents):
    parent1, parent2 = parents
    child = Timetable()
    crossover_point = random.randint(0, len(parent1.genes) - 1)
    child.genes = parent1.genes[:crossover_point] + parent2.genes[crossover_point:]
    child.binary_genes = parent1.binary_genes[:crossover_point] + parent2.binary_genes[crossover_point:]
    return child

def mutate(timetable, mutation_rate=0.1):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    if random.random() < mutation_rate:
        gene_to_mutate_index = random.randint(0, len(timetable.genes) - 1)
        gene = timetable.genes[gene_to_mutate_index]
        new_day = random.choice(days)
        gene.first_lecture_day = new_day
        day_index = days.index(new_day)
        new_binary_day = f'{day_index:03b}'
        binary_gene = timetable.binary_genes[gene_to_mutate_index]
        timetable.binary_genes[gene_to_mutate_index] = binary_gene[:17] + new_binary_day + binary_gene[20:]
    return timetable

def evaluate_population_fitness(population):
    for timetable in population:
        timetable.fitness = timetable.calculate_fitness()

def genetic_algorithm(population_size, num_generations):
    population = initialize_population(population_size)
    evaluate_population_fitness(population)
    for _ in range(num_generations):
        sorted_population = sorted(population, key=lambda t: t.fitness, reverse=True)
        elitism_index = int(0.1 * population_size)
        new_population = sorted_population[:elitism_index]
        while len(new_population) < population_size:
            parents = select_parents(sorted_population)
            child = crossover(parents)
            child = mutate(child)
            child.fitness = child.calculate_fitness()
            new_population.append(child)
        population = new_population
        evaluate_population_fitness(population)
    best_timetable = max(population, key=lambda t: t.fitness)
    print_timetable(best_timetable)
    return best_timetable

def print_timetable(timetable):
    print("Best Timetable with Fitness:", timetable.fitness)
    for gene in timetable.genes:
        print(gene)

best_timetable = genetic_algorithm(population_size=100, num_generations=50)


Generated Timetable:
Binary Gene: 0000010110111001100101110010101111000010111010010111100
Binary Gene: 00110101110000100100100110101111100001100111000001111000
Binary Gene: 0101011110011100010001011100111100000001101111011111000
Binary Gene: 01100100111100110011100001001111000110001001100111100
Binary Gene: 1000100110000100100101010010101111000101010111110111100
Generated Timetable:
Binary Gene: 0000011111000010000110110100101111001001010101010111100
Binary Gene: 001010001011010100110011110111111001000110001101111100
Binary Gene: 0100001110111001001001010001101111000001011001010111100
Binary Gene: 0111011101000001100101011010111100000100000100111111000
Binary Gene: 1001010110001001001001100100111100000100000001011111000
Generated Timetable:
Binary Gene: 0001010100101010101001100011111100001010010001011111000
Binary Gene: 0010101100001001100110010011011111001000111001100111100
Binary Gene: 010100110001110000100101001010111100001001010011001111000
Binary Gene: 011010001100100100000011000